In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier
import lightgbm as lgb

In [7]:
train_df = pd.read_csv(r'C:\Users\sakka\OneDrive\Documents\AMEX\testing.csv')
testing = pd.read_csv(r'C:\Users\sakka\OneDrive\Documents\AMEX\testing.csv')
uteam_id = pd.read_csv(r"C:\Users\sakka\OneDrive\Documents\AMEX\unique_team_features.csv")
ubat_id = pd.read_csv(r'C:\Users\sakka\OneDrive\Documents\AMEX\unique_batsman_features.csv')

In [182]:
ubowl_id = pd.read_csv(r"C:\Users\sakka\OneDrive\Documents\AMEX\unique_bowler_features.csv")

In [109]:
test_df = pd.read_csv(r"C:\Users\sakka\OneDrive\Documents\AMEX\test_updated1.csv")

In [138]:
sub_df = pd.read_csv(r"C:\Users\sakka\OneDrive\Documents\AMEX\submission_template_file1.csv")

In [170]:
sub2_df = pd.read_csv(r"C:\Users\sakka\OneDrive\Documents\AMEX\submitted file\submission_template_file2.csv")

# Feature Engineering

In [9]:
#Encoding for toss and win
train_df['team1_win'] = train_df.apply(lambda row: 1 if row['winner_id'] == row['team1_id'] else 0, axis=1)
train_df['team1_toss_win'] = train_df.apply(lambda row: 1 if row['toss_winner_id'] == row['team1_id'] else 0, axis=1)

## Batting Features

### Average Strike Rate of Team

In [28]:
# Convert the roster IDs from strings to floats for mapping
testing['team1_bat_roster_ids'] = testing['team1_bat_roster_ids'].apply(lambda x: list(map(float, x)))
testing['team2_bat_roster_ids'] = testing['team2_bat_roster_ids'].apply(lambda x: list(map(float, x)))

# Create a dictionary for quick lookup of batting indexes
sr_dict = ubat_id.set_index('batsman_id')['avg_strike_rate'].to_dict()

# Function to calculate the average batting index for a given roster
def calculate_average_strike_rate(roster_ids):
    if not roster_ids:
        return 0
    total_sr = sum(sr_dict.get(player_id, 0) for player_id in roster_ids)
    return total_sr / len(roster_ids)

# Apply the function to each team's roster to get the average batting strength
testing['team1_sr'] = testing['team1_bat_roster_ids'].apply(calculate_average_strike_rate)
testing['team2_sr'] = testing['team2_bat_roster_ids'].apply(calculate_average_strike_rate)

In [34]:
testing['sr_ratio'] = testing['team1_sr']/testing['team2_sr']

### Average Player Runs of Team


In [65]:
# Convert the roster IDs from strings to floats for mapping
testing['team1_bat_roster_ids'] = testing['team1_bat_roster_ids'].apply(lambda x: list(map(float, x)))
testing['team2_bat_roster_ids'] = testing['team2_bat_roster_ids'].apply(lambda x: list(map(float, x)))

# Create a dictionary for quick lookup of batting indexes
runs_dict = ubat_id.set_index('batsman_id')['avg_runs'].to_dict()

# Function to calculate the average batting index for a given roster
def calculate_average_runs(roster_ids):
    if not roster_ids:
        return 0
    total_runs = sum(runs_dict.get(player_id, 0) for player_id in roster_ids)
    return total_runs / len(roster_ids)

# Apply the function to each team's roster to get the average batting strength
testing['team1_player_runs'] = testing['team1_bat_roster_ids'].apply(calculate_average_runs)
testing['team2_player_runs'] = testing['team2_bat_roster_ids'].apply(calculate_average_runs)

In [36]:
testing['avg_player_runs_ratio'] = testing['team1_runs']/testing['team2_runs']

### Average Boundary Percentage of Players

In [17]:
ubat_id = pd.read_csv(r'C:\Users\sakka\OneDrive\Documents\AMEX\unique_batsman_features.csv')
testing = pd.read_csv(r"C:\Users\sakka\OneDrive\Documents\AMEX\testing.csv")

# Split the roster ID strings into lists of individual IDs
testing['team1_bat_roster_ids'] = testing['team1_bat_roster_ids'].str.split(':')
testing['team2_bat_roster_ids'] = testing['team2_bat_roster_ids'].str.split(':')

# Convert the roster IDs from strings to floats for mapping
testing['team1_bat_roster_ids'] = testing['team1_bat_roster_ids'].apply(lambda x: list(map(float, x)))
testing['team2_bat_roster_ids'] = testing['team2_bat_roster_ids'].apply(lambda x: list(map(float, x)))

# Create a dictionary for quick lookup of batting indexes
boundaryp_dict = ubat_id.set_index('batsman_id')['avg_boundary_p'].to_dict()

# Function to calculate the average batting index for a given roster
def calculate_average_bp(roster_ids):
    if not roster_ids:
        return 0
    total_boundary_p = sum(boundaryp_dict.get(player_id, 0) for player_id in roster_ids)
    return total_boundary_p / len(roster_ids)

# Apply the function to each team's roster to get the average batting strength
testing['team1_avg_bp'] = testing['team1_bat_roster_ids'].apply(calculate_average_bp)
testing['team2_avg_bp'] = testing['team2_bat_roster_ids'].apply(calculate_average_bp)

In [37]:
testing['avg_bp_ratio'] = testing['team1_avg_bp']/testing['team2_avg_bp']

In [43]:
#infinite ratio problem
(testing['avg_bp_ratio'] > 50).sum()

1

In [45]:
testing.loc[testing['avg_bp_ratio'] > 50, 'avg_bp_ratio'] = 2

### Total Runs of the players of Team

In [31]:
# Convert the roster IDs from strings to floats for mapping
testing['team1_bat_roster_ids'] = testing['team1_bat_roster_ids'].apply(lambda x: list(map(float, x)))
testing['team2_bat_roster_ids'] = testing['team2_bat_roster_ids'].apply(lambda x: list(map(float, x)))

# Create a dictionary for quick lookup of batting indexes
truns_dict = ubat_id.set_index('batsman_id')['total_runs'].to_dict()

# Function to calculate the average batting index for a given roster
def calculate_average_truns(roster_ids):
    if not roster_ids:
        return 0
    total_truns = sum(truns_dict.get(player_id, 0) for player_id in roster_ids)
    return total_truns / len(roster_ids)

# Apply the function to each team's roster to get the average batting strength
testing['team1_avg_truns'] = testing['team1_bat_roster_ids'].apply(calculate_average_truns)
testing['team2_avg_truns'] = testing['team2_bat_roster_ids'].apply(calculate_average_truns)

In [39]:
testing['avg_truns_ratio'] = testing['team1_avg_truns']/testing['team2_avg_truns']

### Avg Batting Index of the Team

In [193]:
# Convert the roster IDs from strings to floats for mapping
testing['team1_bat_roster_ids'] = testing['team1_bat_roster_ids'].apply(lambda x: list(map(float, x)))
testing['team2_bat_roster_ids'] = testing['team2_bat_roster_ids'].apply(lambda x: list(map(float, x)))

# Create a dictionary for quick lookup of batting indexes
bat_idx_dict = ubat_id.set_index('batsman_id')['batting_index2'].to_dict()

# Function to calculate the average batting index for a given roster
def calculate_average_bat_idx(roster_ids):
    if not roster_ids:
        return 0
    total_bat_idxs = sum(bat_idx_dict.get(player_id, 0) for player_id in roster_ids)
    return total_bat_idxs / len(roster_ids)

# Apply the function to each team's roster to get the average batting strength
testing['team1_avg_bat_idx'] = testing['team1_bat_roster_ids'].apply(calculate_average_bat_idx)
testing['team2_avg_bat_idx'] = testing['team2_bat_roster_ids'].apply(calculate_average_bat_idx)

In [194]:
testing['bat_idx_ratio'] = testing['team1_avg_bat_idx']/testing['team2_avg_bat_idx']

### Avg Total Boundaries of the players in the Team

In [32]:
# Convert the roster IDs from strings to floats for mapping
testing['team1_bat_roster_ids'] = testing['team1_bat_roster_ids'].apply(lambda x: list(map(float, x)))
testing['team2_bat_roster_ids'] = testing['team2_bat_roster_ids'].apply(lambda x: list(map(float, x)))

# Create a dictionary for quick lookup of batting indexes
tb_dict = ubat_id.set_index('batsman_id')['total_scaled_boundaries'].to_dict()

# Function to calculate the average batting index for a given roster
def calculate_average_tb(roster_ids):
    if not roster_ids:
        return 0
    total_tb = sum(tb_dict.get(player_id, 0) for player_id in roster_ids)
    return total_tb / len(roster_ids)

# Apply the function to each team's roster to get the average batting strength
testing['team1_avg_tb'] = testing['team1_bat_roster_ids'].apply(calculate_average_tb)
testing['team2_avg_tb'] = testing['team2_bat_roster_ids'].apply(calculate_average_tb)

In [40]:
testing['avg_tb_ratio'] = testing['team1_avg_tb']/testing['team2_avg_tb']

In [47]:
#infinite ratio problem
(testing['avg_tb_ratio'] > 50).sum()

1

In [48]:
testing.loc[testing['avg_tb_ratio'] > 50, 'avg_tb_ratio'] = 2

## Bowling Features

### Average Economy

### Loading Team features 

In [73]:
#for team1
testing = testing.merge(uteam_id, how='left', left_on='team1_id', right_on='team_id')

# Rename the team_runs column to team1_runs and drop the team_id column
testing.rename(columns={'team_runs': 'team1_runs','team_wickets': 'team1_wickets','avg_rr': 'team1_avg_rr','winp':'team1_winp',
                        'avg_win_runs':'team1_avg_win_runs','avg_win_wkts':'team1_avg_win_wkts'}, inplace=True)
testing.drop(columns=['team_id'], inplace=True)

In [74]:
#for team2
testing = testing.merge(uteam_id, how='left', left_on='team2_id', right_on='team_id')

# Rename the team_runs column to team1_runs and drop the team_id column
testing.rename(columns={'team_runs': 'team2_runs','team_wickets': 'team2_wickets','avg_rr': 'team2_avg_rr','winp':'team2_winp',
                        'avg_win_runs':'team2_avg_win_runs','avg_win_wkts':'team2_avg_win_wkts'}, inplace=True)
testing.drop(columns=['team_id'], inplace=True)

In [75]:
#Adding the ratios

#team_runs_ratio
testing['team_runs_ratio'] = testing['team1_runs'] / testing['team2_runs']
testing['team_runs_ratio'] = np.where(testing['team1_runs'].isna() | testing['team2_runs'].isna(), np.nan, testing['team_runs_ratio'])

#team winp_ratio
testing['winp_ratio'] = testing['team1_winp'] / testing['team2_winp']
testing['winp_ratio'] = np.where(testing['team1_winp'].isna() | testing['team2_winp'].isna(), np.nan, testing['winp_ratio'])

#team win_runs_ratio
testing['win_runs_ratio'] = testing['team1_avg_win_runs'] / testing['team2_avg_win_runs']
testing['win_runs_ratio'] = np.where(testing['team1_avg_win_runs'].isna() | testing['team2_avg_win_runs'].isna(), np.nan, testing['win_runs_ratio'])

#team win_wkts_ratio
testing['win_wkts_ratio'] = testing['team1_avg_win_wkts'] / testing['team2_avg_win_wkts']
testing['win_wkts_ratio'] = np.where(testing['team1_avg_win_wkts'].isna() | testing['team2_avg_win_wkts'].isna(), np.nan, testing['win_wkts_ratio'])

#team rr_ratio
testing['rr_ratio'] = testing['team1_avg_rr'] / testing['team2_avg_rr']
testing['rr_ratio'] = np.where(testing['team1_avg_rr'].isna() | testing['team2_avg_rr'].isna(), np.nan, testing['rr_ratio'])

#team wkt_ratio
testing['wkt_ratio'] = testing['team2_wickets'] / testing['team1_wickets']
testing['wkt_ratio'] = np.where(testing['team1_wickets'].isna() | testing['team2_wickets'].isna(), np.nan, testing['wkt_ratio'])

In [196]:
testing.columns.to_list()

['match_id',
 'team1_id',
 'team1_roster_ids',
 'team2_id',
 'team2_roster_ids',
 'winner_id',
 'venue',
 'city',
 'series_name',
 'ground_id',
 'team_count_50runs_last15',
 'team_winp_last5',
 'team1only_avg_runs_last15',
 'team1_winp_team2_last15',
 'ground_avg_runs_last15',
 'year',
 'toss_decision_bat',
 'toss_decision_field',
 'toss_winner_id',
 'team1_bat_roster_ids',
 'team2_bat_roster_ids',
 'team1_bowl_roster_ids',
 'team2_bowl_roster_ids',
 'team1_avg_bp',
 'team2_avg_bp',
 'team1_sr',
 'team2_sr',
 'team1_avg_truns',
 'team2_avg_truns',
 'team1_avg_tb',
 'team2_avg_tb',
 'sr_ratio',
 'avg_player_runs_ratio',
 'avg_bp_ratio',
 'avg_truns_ratio',
 'avg_tb_ratio',
 'team1_runs',
 'team1_wickets',
 'team1_avg_rr',
 'team1_winp',
 'team1_avg_win_runs',
 'team1_avg_win_wkts',
 'team2_runs',
 'team2_wickets',
 'team2_avg_rr',
 'team2_winp',
 'team2_avg_win_runs',
 'team2_avg_win_wkts',
 'team_runs_ratio',
 'winp_ratio',
 'win_runs_ratio',
 'win_wkts_ratio',
 'rr_ratio',
 'wkt_ratio

In [77]:
#drop the unnecessary columns
testing.drop(columns = ['team_balls_x','avg_overs_x','matches_x','wins_x','total_win_runs_x','total_win_wkts_x','match_win_runs_x','match_win_wkts_x',
                        'team_balls_y','avg_overs_y','matches_y','wins_y','total_win_runs_y','total_win_wkts_y','match_win_runs_y','match_win_wkts_y'], 
             inplace = True)

### Merge with the train_df

In [97]:
columns_to_merge = ['match_id','sr_ratio','avg_player_runs_ratio','avg_bp_ratio','avg_truns_ratio','avg_tb_ratio','team_runs_ratio','winp_ratio','win_runs_ratio',
                    'win_wkts_ratio','rr_ratio','wkt_ratio']

# Merge the DataFrames on 'match_id'
train_df = pd.merge(train_df, testing[columns_to_merge], on='match_id', how='left')

In [108]:
train_df.columns.to_list()

['match_id',
 'team1_id',
 'team1_roster_ids',
 'team2_id',
 'team2_roster_ids',
 'winner_id',
 'venue',
 'city',
 'series_name',
 'ground_id',
 'team_count_50runs_last15',
 'team_winp_last5',
 'team1only_avg_runs_last15',
 'team1_winp_team2_last15',
 'ground_avg_runs_last15',
 'year',
 'toss_decision_bat',
 'toss_decision_field',
 'toss_winner_id',
 'team1_bat_roster_ids',
 'team2_bat_roster_ids',
 'team1_bowl_roster_ids',
 'team2_bowl_roster_ids',
 'team1_win',
 'team1_toss_win',
 'sr_ratio',
 'avg_player_runs_ratio',
 'avg_bp_ratio',
 'avg_truns_ratio',
 'avg_tb_ratio',
 'team_runs_ratio',
 'winp_ratio',
 'win_runs_ratio',
 'win_wkts_ratio',
 'rr_ratio',
 'wkt_ratio',
 'team1_win_pred',
 'win_pred_team_id',
 'win_pred_score']

## Applying Catboost Model

In [103]:
X = train_df.drop(columns=['match_id','team1_id','team1_roster_ids','team2_id','team2_roster_ids','winner_id','venue','city','series_name','ground_id',
                           'team1only_avg_runs_last15','ground_avg_runs_last15','year','toss_decision_bat','toss_decision_field','toss_winner_id',
                           'team1_bat_roster_ids','team2_bat_roster_ids','team1_bowl_roster_ids','team2_bowl_roster_ids','team1_win'])
y = train_df['team1_win']  # Target variable

# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, loss_function='Logloss')

# Train the model
model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7052631578947368


In [104]:
#Adding predicted team1_win column
y_train_pred = model.predict(X)
train_df['team1_win_pred'] = y_train_pred

In [105]:
#Add win_pred_team_id

# Function to determine the winning team_id based on team1_win_pred
def determine_winner(row):
    return row['team1_id'] if row['team1_win_pred'] == 1 else row['team2_id']

# Apply the function to create the win_pred_team_id column
train_df['win_pred_team_id'] = train_df.apply(determine_winner, axis=1)

In [106]:
#Add win_pred_score of the team

# Predict probabilities for each class (0 and 1)
proba_preds1 = model.predict_proba(X)

# Get probability of team1 winning (assuming class index 1 corresponds to team1_win_pred)
win_pred_scores1 = proba_preds1[:, 1]

# Add win_pred_score column to DataFrame
train_df['win_pred_score'] = win_pred_scores1

In [107]:
train_df['win_pred_score'] = train_df['win_pred_score'].apply(lambda x: x if x >= 0.5 else 1 - x)

## Test Data

In [114]:
test_df.rename(columns = {'avg_runs_ratio':'avg_player_runs_ratio','bp_ratio':'avg_bp_ratio','truns_ratio':'avg_truns_ratio',
                          't_boundary_ratio':'avg_tb_ratio'}, inplace = True)

In [124]:
#Apply the model

In [119]:
train_columns = ['team_count_50runs_last15','team_winp_last5','team1_winp_team2_last15','team1_toss_win','sr_ratio',
                'avg_player_runs_ratio','avg_bp_ratio','avg_truns_ratio','avg_tb_ratio',
                 'team_runs_ratio','winp_ratio','win_runs_ratio','win_wkts_ratio','rr_ratio','wkt_ratio']

X_test_df = test_df.drop(columns=['match_id','team1','team1_id','team1_roster_ids','team2','team2_id','team2_roster_ids','toss winner','toss decision',
                                  'venue','city','match_dt','lighting','series_name','season','ground_id','team1only_avg_runs_last15',
                                  'ground_avg_runs_last15'])

# Reorder columns to match the training data
X_test_df = X_test_df[train_columns]

y_test_pred = model.predict(X_test_df)

# Add the predictions to the test dataframe
test_df['team1_win_pred'] = y_test_pred

In [120]:
#win Prediction team_id

# Function to determine the winning team_id based on team1_win_pred
def determine_winner(row):
    return row['team1_id'] if row['team1_win_pred'] == 1 else row['team2_id']

# Apply the function to create the win_pred_team_id column
test_df['win_pred_team_id'] = test_df.apply(determine_winner, axis=1)

In [121]:
#win prediction score

# Predict probabilities for each class (0 and 1)
proba_preds = model.predict_proba(X_test_df)

# Get probability of team1 winning (assuming class index 1 corresponds to team1_win_pred)
win_pred_scores = proba_preds[:, 1]

# Add win_pred_score column to DataFrame
test_df['win_pred_score'] = win_pred_scores

In [122]:
test_df['win_pred_score'] = test_df['win_pred_score'].apply(lambda x: x if x >= 0.5 else 1 - x)

In [132]:
#top 10 features for train and test

In [128]:
features = X_test_df.columns[:15]

# Give an id numbering from 1 to 15 to the fifteen features
feature_ids = {feature: i+1 for i, feature in enumerate(features)}

# Get feature importances
feature_importances = model.get_feature_importance(prettified=True)

# Create a dictionary with feature names as keys and their importances as values
importances = {row['Feature Id']: row['Importances'] for _, row in feature_importances.iterrows()}

# Sort features based on their importances
sorted_features = sorted(importances.items(), key=lambda x: x[1], reverse=True)

# Create 10 columns of top 10 features i.e. top_feature1, top_feature2 and so on
for i in range(10):
    train_df[f'top_feature{i+1}'] = sorted_features[i][0]

# For each row, add the id of the top ten features in their corresponding column
for i in range(10):
    train_df[f'top_feature{i+1}'] = train_df[f'top_feature{i+1}'].map(feature_ids)


In [133]:
for i in range(10):
    test_df[f'top_feature{i+1}'] = sorted_features[i][0]

# For each row, add the id of the top ten features in their corresponding column
for i in range(10):
    test_df[f'top_feature{i+1}'] = test_df[f'top_feature{i+1}'].map(feature_ids)

In [130]:
# Print the feature_ids dictionary
print("Feature IDs:")
for feature, id in feature_ids.items():
    print(f"{id}: {feature}")

Feature IDs:
1: team_count_50runs_last15
2: team_winp_last5
3: team1_winp_team2_last15
4: team1_toss_win
5: sr_ratio
6: avg_player_runs_ratio
7: avg_bp_ratio
8: avg_truns_ratio
9: avg_tb_ratio
10: team_runs_ratio
11: winp_ratio
12: win_runs_ratio
13: win_wkts_ratio
14: rr_ratio
15: wkt_ratio


## The submission file

In [156]:
#fill with the train data
id_mapping = train_df.set_index('match_id')['win_pred_team_id'].to_dict()

# Fill missing values in sub_df['win_pred_team_id'] using the mapping
sub_df['win_pred_team_id'] = sub_df['match_id'].map(id_mapping).fillna(sub_df['win_pred_team_id'])

In [157]:
#fill with the train data
id_mapping = train_df.set_index('match_id')['win_pred_score'].to_dict()

# Fill missing values in sub_df['win_pred_team_id'] using the mapping
sub_df['win_pred_score'] = sub_df['match_id'].map(id_mapping).fillna(sub_df['win_pred_score'])

In [158]:
#fill with test data
id_mapping = test_df.set_index('match_id')['win_pred_team_id'].to_dict()

# Fill missing values in sub_df['win_pred_team_id'] using the mapping
sub_df['win_pred_team_id'] = sub_df['match_id'].map(id_mapping).fillna(sub_df['win_pred_team_id'])

In [159]:
id_mapping = test_df.set_index('match_id')['win_pred_score'].to_dict()

# Fill missing values in sub_df['win_pred_team_id'] using the mapping
sub_df['win_pred_score'] = sub_df['match_id'].map(id_mapping).fillna(sub_df['win_pred_score'])

In [166]:
sub_df.head()

,match id,dataset_type,win_pred_team_id,win_pred_score,train_algorithm,is_ensemble,train_hps_trees,train_hps_depth,train_hps_lr,indep_feat_id1,indep_feat_id2,indep_feat_id3,indep_feat_id4,indep_feat_id5,indep_feat_id6,indep_feat_id7,indep_feat_id8,indep_feat_id9,indep_feat_id10
0,9250275,r1,28594.0,0.541319,catboost,no,1000.0,6.0,0.1,11.0,15.0,13.0,8.0,14.0,2.0,7.0,10.0,12.0,1.0
1,9262189,r1,36084.0,0.667763,catboost,no,1000.0,6.0,0.1,11.0,15.0,13.0,8.0,14.0,2.0,7.0,10.0,12.0,1.0
2,9128776,r1,48334.0,0.555825,catboost,no,1000.0,6.0,0.1,11.0,15.0,13.0,8.0,14.0,2.0,7.0,10.0,12.0,1.0
3,9586919,r1,36098.0,0.524820,catboost,no,1000.0,6.0,0.1,11.0,15.0,13.0,8.0,14.0,2.0,7.0,10.0,12.0,1.0
4,9128538,r1,48341.0,0.545569,catboost,no,1000.0,6.0,0.1,11.0,15.0,13.0,8.0,14.0,2.0,7.0,10.0,12.0,1.0


In [165]:
sub_df['train_algorithm'] = sub_df['train_algorithm'].fillna('catboost')
sub_df['is_ensemble'] = sub_df['is_ensemble'].fillna('no')
sub_df['train_hps_depth'] = sub_df['train_hps_depth'].fillna(6)
sub_df['train_hps_trees'] = sub_df['train_hps_trees'].fillna(1000)
sub_df['train_hps_lr'] = sub_df['train_hps_lr'].fillna(0.1)
sub_df['indep_feat_id1'] = sub_df['indep_feat_id1'].fillna(11)
sub_df['indep_feat_id2'] = sub_df['indep_feat_id2'].fillna(15)
sub_df['indep_feat_id3'] = sub_df['indep_feat_id3'].fillna(13)
sub_df['indep_feat_id4'] = sub_df['indep_feat_id4'].fillna(8)
sub_df['indep_feat_id5'] = sub_df['indep_feat_id5'].fillna(14)
sub_df['indep_feat_id6'] = sub_df['indep_feat_id6'].fillna(2)
sub_df['indep_feat_id7'] = sub_df['indep_feat_id7'].fillna(7)
sub_df['indep_feat_id8'] = sub_df['indep_feat_id8'].fillna(10)
sub_df['indep_feat_id9'] = sub_df['indep_feat_id9'].fillna(12)
sub_df['indep_feat_id10'] = sub_df['indep_feat_id10'].fillna(1)

In [168]:
feature_importances = model.feature_importances_

# Create a dataframe to store feature importances
importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
})

# Sort by importance (descending order)
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Print the feature importance rank
print(importance_df)

                     Feature  Importance
10                winp_ratio   16.530527
14                 wkt_ratio    8.600006
12            win_wkts_ratio    8.154526
7            avg_truns_ratio    7.514163
13                  rr_ratio    7.452120
1            team_winp_last5    6.944660
6               avg_bp_ratio    6.828447
9            team_runs_ratio    6.236851
11            win_runs_ratio    6.208107
0   team_count_50runs_last15    5.776017
4                   sr_ratio    4.672838
8               avg_tb_ratio    4.495505
2    team1_winp_team2_last15    4.265701
5      avg_player_runs_ratio    4.036772
3             team1_toss_win    2.283760


In [206]:
sub_df.columns.to_list()

['match id',
 'dataset_type',
 'win_pred_team_id',
 'win_pred_score',
 'train_algorithm',
 'is_ensemble',
 'train_hps_trees',
 'train_hps_depth',
 'train_hps_lr',
 'indep_feat_id1',
 'indep_feat_id2',
 'indep_feat_id3',
 'indep_feat_id4',
 'indep_feat_id5',
 'indep_feat_id6',
 'indep_feat_id7',
 'indep_feat_id8',
 'indep_feat_id9',
 'indep_feat_id10']

In [169]:
sub_df.to_csv(r'C:\Users\sakka\OneDrive\Documents\AMEX\submitted file\subm1.csv')

In [174]:
importance_df.rename(columns = {'Feature':'feat_name'}, inplace = True)

In [176]:
importance_df.rename(columns = {'Importance':'model_feat_imp_train'}, inplace = True)

In [187]:
importance_df

,feat_name,model_feat_imp_train
10,winp_ratio,16.530527
14,wkt_ratio,8.600006
12,win_wkts_ratio,8.154526
7,avg_truns_ratio,7.514163
13,rr_ratio,7.452120
1,team_winp_last5,6.944660
6,avg_bp_ratio,6.828447
9,team_runs_ratio,6.236851
11,win_runs_ratio,6.208107
0,team_count_50runs_last15,5.776017


In [204]:
sub2_df.to_csv(r"C:\Users\sakka\OneDrive\Documents\AMEX\submitted file\submission_file2.csv")

In [178]:
#fill with test data
id_mapping = importance_df.set_index('feat_name')['model_feat_imp_train'].to_dict()

# Fill missing values in sub_df['win_pred_team_id'] using the mapping
sub2_df['model_feat_imp_train'] = sub2_df['feat_name'].map(id_mapping).fillna(sub2_df['model_feat_imp_train'])

In [189]:
train_df.to_csv(r'C:\Users\sakka\OneDrive\Documents\AMEX\train_after_sub1.csv', index = False)

In [192]:
ubat_id.head()

,batsman_id,batting_index2,avg_strike_rate,avg_runs,total_runs,avg_boundary_p,total_scaled_boundaries,not_out
0,31464.0,0.383086,40.910000,3.333333,10,0.000000,0.0,0.0
1,34061.0,4.835179,110.679259,12.555556,678,0.652375,65.0,18.0
2,36665.0,5.376091,99.854000,20.800000,208,0.637143,11.0,1.0
3,37351.0,7.270954,106.113226,25.645161,1590,0.636146,153.5,6.0
4,41740.0,6.431870,114.373409,22.977273,1011,0.573818,77.5,10.0


In [ ]:
#fill with test data
id_mapping = test_df.set_index('match_id')['win_pred_team_id'].to_dict()

# Fill missing values in sub_df['win_pred_team_id'] using the mapping
sub_df['win_pred_team_id'] = sub_df['match_id'].map(id_mapping).fillna(sub_df['win_pred_team_id'])

In [205]:
train_df.columns.to_list()

['match_id',
 'team1_id',
 'team1_roster_ids',
 'team2_id',
 'team2_roster_ids',
 'winner_id',
 'venue',
 'city',
 'series_name',
 'ground_id',
 'team_count_50runs_last15',
 'team_winp_last5',
 'team1only_avg_runs_last15',
 'team1_winp_team2_last15',
 'ground_avg_runs_last15',
 'year',
 'toss_decision_bat',
 'toss_decision_field',
 'toss_winner_id',
 'team1_bat_roster_ids',
 'team2_bat_roster_ids',
 'team1_bowl_roster_ids',
 'team2_bowl_roster_ids',
 'team1_win',
 'team1_toss_win',
 'sr_ratio',
 'avg_player_runs_ratio',
 'avg_bp_ratio',
 'avg_truns_ratio',
 'avg_tb_ratio',
 'team_runs_ratio',
 'winp_ratio',
 'win_runs_ratio',
 'win_wkts_ratio',
 'rr_ratio',
 'wkt_ratio',
 'team1_win_pred',
 'win_pred_team_id',
 'win_pred_score',
 'top_feature1',
 'top_feature2',
 'top_feature3',
 'top_feature4',
 'top_feature5',
 'top_feature6',
 'top_feature7',
 'top_feature8',
 'top_feature9',
 'top_feature10']

In [198]:
columns_to_merge = ['match_id','bat_idx_ratio']

# Merge the DataFrames on 'match_id'
train_df = pd.merge(train_df, testing[columns_to_merge], on='match_id', how='left')

In [199]:
filtered_df = train_df[train_df['bat_idx_ratio']> 1]

# Step 2: Count rows where both col1 > 1 and col2 == 1
count_col1_gt_1_and_col2_1 = filtered_df[filtered_df['team1_win'] == 1].shape[0]

# Step 3: Total number of rows where col1 > 1
total_col1_gt_1 = filtered_df.shape[0]

# Step 4: Calculate the percentage
if total_col1_gt_1 > 0:
    percentage = (count_col1_gt_1_and_col2_1 / total_col1_gt_1) * 100
else:
    percentage = 0.0  # Handle the case where there are no rows where col1 > 1
percentage

56.89277899343544

## team year features analysis

In [202]:
ubat_id.head()

,batsman_id,batting_index2,avg_strike_rate,avg_runs,total_runs,avg_boundary_p,total_scaled_boundaries,not_out
0,31464.0,0.383086,40.910000,3.333333,10,0.000000,0.0,0.0
1,34061.0,4.835179,110.679259,12.555556,678,0.652375,65.0,18.0
2,36665.0,5.376091,99.854000,20.800000,208,0.637143,11.0,1.0
3,37351.0,7.270954,106.113226,25.645161,1590,0.636146,153.5,6.0
4,41740.0,6.431870,114.373409,22.977273,1011,0.573818,77.5,10.0


In [203]:
# Convert match_dt to datetime format
mat_df['match_dt'] = pd.to_datetime(mat_df['match_dt'])

# Extract the year and create a new column
mat_df['year'] = mat_df['match_dt'].dt.year

NameError: name 'mat_df' is not defined